<center><b>DIGHUM101</b></center>
<center>4-4: Google Books API</center>

---

# Collecting Data from the Web - APIs


An API is a connection between computers or between computer programs. It is a type of software interface, offering a service to other pieces of software. The kinds of APIs we will use here can be used to interact with, and get data from, different web platforms using Python. Some APIs are more public than others.

We're going to first explore the Google Books API to perform some searches for books and see what metadata we get in return. Although many APIs require a key in order to access the data, we can perform Google Books searches without one. 

Using APIs to request data is usually done through HTTP (see [here](https://developer.mozilla.org/en-US/docs/Web/HTTP/Overview) for an overview of basic terms). The guide to the Google Books API can be found [here](https://developers.google.com/books/).

First we'll import the [`requests`](http://docs.python-requests.org/en/master/) library. The `requests` library is necessary for most interaction with the internet in Python. We'll use it to make a `get` request to the API endpoint.

In [ ]:
# !pip install requests  

In [1]:
import requests
import pandas as pd
import time

To call an API it suffices to point it to a unique URL. We always need a base URL, or endpoint, to which we can add the parameters specific to our request. Let's assign the base Google Books endpoint to a variable, we'll call it `books_url`. We know this URL from the documentation linked above.

In [2]:
books_url = 'https://www.googleapis.com/books/v1/volumes?'

We can start off with a very simple search to see what the results look like. Then we'll move on to adding more filters and parameters. Let's assign our query to a variable `query`.

In [17]:
query = 'digital humanities'

To incorporate this into our query we can make a dictionary called `parameters`. Read more about what kinds of parameters the Google API allows for [here](https://developers.google.com/books/docs/v1/using#st_params).

We'll pass these parameters to the `get` method. Read more about HTTP GET requests [here](https://www.w3schools.com/tags/ref_httpmethods.asp). 

The `'q'` stands for 'query', and whatever value we assign to that is what we're searching for, just as if we typed it into the Google search bar.

In [18]:
parameters = {'q': query}

We'll pass two arguments to the `get` method of `requests` library: the URL and the parameters we want. It returns a response object.

In [19]:
r = requests.get(books_url, params = parameters)

Printing the `url` property, we can see that this function converted the URL into the proper format to include our search terms.

In [20]:
print(r.url)

https://www.googleapis.com/books/v1/volumes?q=digital+humanities


To see our results, we simply use the request object's `json` method, which we can then navigate in the same way as a Python dictionary. Take a minute or two to navigate through the results in order to see what's there. 

In [21]:
results = r.json()
results

{'kind': 'books#volumes',
 'totalItems': 1000000,
 'items': [{'kind': 'books#volume',
   'id': 'GmImK8hCPjcC',
   'etag': 'J/PD5q7HYAY',
   'selfLink': 'https://www.googleapis.com/books/v1/volumes/GmImK8hCPjcC',
   'volumeInfo': {'title': 'Digital Humanities Pedagogy',
    'subtitle': 'Practices, Principles and Politics',
    'authors': ['Brett D. Hirsch'],
    'publisher': 'Open Book Publishers',
    'publishedDate': '2012',
    'description': '"The essays in this collection offer a timely intervention in digital humanities scholarship, bringing together established and emerging scholars from a variety of humanities disciplines across the world. The first section offers views on the practical realities of teaching digital humanities at undergraduate and graduate levels, presenting case studies and snapshots of the authors\' experiences alongside models for future courses and reflections on pedagogical successes and failures. The next section proposes strategies for teaching foundation

You probably figured out that the books are found under the `items` key, and the most important information for each one is under the `volumeInfo` key. Let's take a look at the first result.

In [22]:
results['items'][0]['volumeInfo']

{'title': 'Digital Humanities Pedagogy',
 'subtitle': 'Practices, Principles and Politics',
 'authors': ['Brett D. Hirsch'],
 'publisher': 'Open Book Publishers',
 'publishedDate': '2012',
 'description': '"The essays in this collection offer a timely intervention in digital humanities scholarship, bringing together established and emerging scholars from a variety of humanities disciplines across the world. The first section offers views on the practical realities of teaching digital humanities at undergraduate and graduate levels, presenting case studies and snapshots of the authors\' experiences alongside models for future courses and reflections on pedagogical successes and failures. The next section proposes strategies for teaching foundational digital humanities methods across a variety of scholarly disciplines, and the book concludes with wider debates about the place of digital humanities in the academy, from the field\'s cultural assumptions and social obligations to its politi

There's a lot of information in the results, but we probably don't want all of it. Suppose that for each volume in the results, we only want to extract 1) the title, 2) the author(s), 3) the publication date, and 4) the description. Below is a function named `parse_results` that takes the `results` variable as an argument and returns a list of dictionaries. Each dictionary within the list corresponds to a book, and has an `author` key, a `title` key, a `publication_date` key, and a `description` key.

In [35]:
def parse_results(results):
    results_list = []

    # Only process if 'items' is present
    if 'items' not in results:
        #print("No items found on this page.")
        return results_list

    for book in results['items']:
        title = book['volumeInfo']['title']

        # some books don't have authors, dates, or a description
        try:
            authors = ','.join(book['volumeInfo']['authors'])
        except:
            authors = 'NA'
        
        try:
            published_date = book['volumeInfo']['publishedDate']
        except:
            published_date = 'NA'

        try:
            description = book['volumeInfo']['description']
        except:
            description = "NA"

        results_dict = {'title': title,
                        'authors': authors,
                        'description': description,
                        'published_date': published_date}
        
        results_list.append(results_dict)
        
    return(results_list)

In [28]:
# Use our function to parse the results
data = parse_results(results)
data

[{'title': 'Digital Humanities Pedagogy',
  'authors': 'Brett D. Hirsch',
  'description': '"The essays in this collection offer a timely intervention in digital humanities scholarship, bringing together established and emerging scholars from a variety of humanities disciplines across the world. The first section offers views on the practical realities of teaching digital humanities at undergraduate and graduate levels, presenting case studies and snapshots of the authors\' experiences alongside models for future courses and reflections on pedagogical successes and failures. The next section proposes strategies for teaching foundational digital humanities methods across a variety of scholarly disciplines, and the book concludes with wider debates about the place of digital humanities in the academy, from the field\'s cultural assumptions and social obligations to its political visions." (4e de couverture).',
  'published_date': '2012'},
 {'title': 'Disrupting the Digital Humanities',
 

In [25]:
# Convert the results into a dataframe
results_df = pd.DataFrame(data)
results_df

,title,authors,description,published_date
0,Digital Humanities Pedagogy,Brett D. Hirsch,"""The essays in this collection offer a timely ...",2012
1,Disrupting the Digital Humanities,"Dorothy Kim,Jesse Stommel","All too often, defining a discipline becomes m...",2018
2,Interdisciplining Digital Humanities,Julie Thompson Klein,The first book to test the claim that the emer...,2015-01-05
3,Digital Humanities,"David M. Berry,Anders Fagerjord","As the twenty-first century unfolds, computers...",2017-05-30
4,Defining Digital Humanities,"Melissa Terras,Julianne Nyhan,Edward Vanhoutte",Digital Humanities is becoming an increasingly...,2016-05-13
5,Digital Humanities in Practice,"Claire Warwick,Melissa Terras,Julianne Nyhan",This cutting-edge and comprehensive introducti...,2012-10-09
6,Doing Digital Humanities,"Constance Crompton,Richard Lane,Ray Siemens",Digital Humanities is rapidly evolving as a si...,2016-09-13
7,Digital_Humanities,"Anne Burdick,Johanna Drucker,Peter Lunenfeld,T...",A visionary report on the revitalization of th...,2016-02-12
8,The Digital Humanities Coursebook,Johanna Drucker,"""The Digital Humanities Coursebook provides cr...",2021
9,Debates in the Digital Humanities 2016,"Matthew K. Gold,Lauren F. Klein",Pairing full-length scholarly essays with shor...,2016-05-18


Now let's explore the API using more parameters. You may have noticed that our query only gave us 10 books, but there are probably more than 10 books written about digital humanities. To adjust our search, we need to add in the `maxResults` parameter and the `startIndex` parameter. We can do that by adding these as keys to the `parameters` dictionary, and then run our request again. To read about these parameters, see the [documentation](https://developers.google.com/books/docs/v1/using#api_params).

In [29]:
parameters = {'q': query,
              'startIndex': 0,
              'maxResults': 10} # max 40 per request, as per documentation

r = requests.get(books_url, params = parameters)

print(r.url)

results = r.json()

print()

parse_results(results)

https://www.googleapis.com/books/v1/volumes?q=digital+humanities&startIndex=0&maxResults=10



[{'title': 'Digital Humanities Pedagogy',
  'authors': 'Brett D. Hirsch',
  'description': '"The essays in this collection offer a timely intervention in digital humanities scholarship, bringing together established and emerging scholars from a variety of humanities disciplines across the world. The first section offers views on the practical realities of teaching digital humanities at undergraduate and graduate levels, presenting case studies and snapshots of the authors\' experiences alongside models for future courses and reflections on pedagogical successes and failures. The next section proposes strategies for teaching foundational digital humanities methods across a variety of scholarly disciplines, and the book concludes with wider debates about the place of digital humanities in the academy, from the field\'s cultural assumptions and social obligations to its political visions." (4e de couverture).',
  'published_date': '2012'},
 {'title': 'Disrupting the Digital Humanities',
 

Now, we can write a for-loop to collect the first 100 results into `all_results`. But make sure you use `time.sleep` at the end of each loop! Python is so fast that if you write a for loop without pausing between calls you can overload someone's server, or get yourself (temporarily) banned:

In [14]:
# See how .sleep works

for x in range(5):
    print(x)
    time.sleep(1) # alter this number to change sleep length

0
1
2
3
4


In [36]:
parameters = {'q': query,
          'maxResults': 20,
          'startIndex': 0}

all_results = []
for i in range(5):
    print("collecting page " + str(i + 1))
    
    r = requests.get(books_url, params=parameters)
    results = r.json()
    if 'items' not in results:
        print(f"Warning: no items found. Full response:\n{r.url}\n{results}")
    parsed = parse_results(results)
    all_results.extend(parsed)
    
    time.sleep(1) # very important to not overload API!!!
    parameters['startIndex'] += parameters['maxResults'] # note the addition assignment 

collecting page 1
collecting page 2
collecting page 3
collecting page 4
https://www.googleapis.com/books/v1/volumes?q=digital+humanities&maxResults=20&startIndex=60
{'kind': 'books#volumes', 'totalItems': 17}
collecting page 5
https://www.googleapis.com/books/v1/volumes?q=digital+humanities&maxResults=20&startIndex=80
{'kind': 'books#volumes', 'totalItems': 12}


In [37]:
print(len(all_results))

60


In [34]:
all_results_df = pd.DataFrame(all_results)
all_results_df.head()

,title,authors,description,published_date
0,Digital Humanities Pedagogy,Brett D. Hirsch,"""The essays in this collection offer a timely ...",2012
1,Disrupting the Digital Humanities,"Dorothy Kim,Jesse Stommel","All too often, defining a discipline becomes m...",2018
2,Interdisciplining Digital Humanities,Julie Thompson Klein,The first book to test the claim that the emer...,2015-01-05
3,Digital Humanities,"David M. Berry,Anders Fagerjord","As the twenty-first century unfolds, computers...",2017-05-30
4,Defining Digital Humanities,"Melissa Terras,Julianne Nyhan,Edward Vanhoutte",Digital Humanities is becoming an increasingly...,2016-05-13


Now we can write this data to a CSV.

In [ ]:
import csv

keys = all_results[0].keys()

with open('books_search.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(all_results)